<a href="https://colab.research.google.com/github/Madhulikabaranwal/Complete-Python-3-Bootcamp/blob/master/R6_Internal_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [38]:
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [39]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/prices.csv')

In [0]:
from google.colab import drive


### Check all columns in the dataset

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [44]:
data.shape

(851264, 7)

### Drop columns `date` and  `symbol`

In [0]:
d=data.drop('date',axis=1)
f=d.drop('symbol',axis=1)

In [46]:
f.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [47]:
A=f.head(1000)
A

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...
995,63.310001,63.590000,63.240002,63.639999,2133200.0
996,27.160000,26.990000,26.680000,27.299999,1982400.0
997,28.320000,28.770000,28.010000,28.809999,37152800.0
998,44.000000,44.799999,43.750000,44.810001,6568600.0


In [48]:
A['volume']=A['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
X=A.drop('volume',axis=1)
y=A[['volume']]

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)



#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np

X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_train = np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)




## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [61]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)))

Current Loss on iteration 0 Tensor("Mean_1:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_3:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_5:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_7:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_9:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_11:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_13:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_15:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_17:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_19:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_21:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_23:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_25:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor("Mean_27:0", shap

### Get the shapes and values of W and b

In [62]:
#Check Weights and Bias
print('Weights:\n', w)
print('Bias:\n',b)

Weights:
 Tensor("sub_397:0", shape=(4, 1), dtype=float32)
Bias:
 Tensor("sub_398:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [0]:
def prediction(X_test, w, b):
    
    xw_matmul = tf.matmul(X_test, w)
    y = tf.add(xw_matmul, b)
    
    return y


In [0]:
with tf.Session() as sess:
  val_2=(prediction(X_test, w, b).eval())

In [67]:
val_2[0]

array([5.4272947], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data1 = pd.read_csv('/gdrive/My Drive/Iris (2).csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [74]:
pd.get_dummies(data1['Species'])

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [0]:
data1=pd.get_dummies(data1)

In [81]:
data1

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,0,0,1
146,147,6.3,2.5,5.0,1.9,0,0,1
147,148,6.5,3.0,5.2,2.0,0,0,1
148,149,6.2,3.4,5.4,2.3,0,0,1


### Splitting the data into feature set and target set

In [0]:
X2 = data1.drop(columns=['Id', 'Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'], axis=1)
Y2 = data1[['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica']]

In [0]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.30, random_state=55)
X2_train_array = X2_train.values
Y2_train_array = Y2_train.values
X2_test_array = X2_test.values
Y2_test_array = Y2_test.values

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(4,activation='sigmoid'))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [88]:
model.fit(X2_train_array, Y2_train_array, 
          validation_data=(X2_test_array,Y2_test_array), 
          epochs=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 4ms/sample - loss: 1.5662 - acc: 0.3238 - val_loss: 1.4703 - val_acc: 0.3556
Epoch 2/100
105/105 [==============================] - 0s 134us/sample - loss: 1.5162 - acc: 0.3238 - val_loss: 1.4282 - val_acc: 0.3556
Epoch 3/100
105/105 [==============================] - 0s 114us/sample - loss: 1.4718 - acc: 0.3238 - val_loss: 1.3898 - val_acc: 0.3556
Epoch 4/100
105/105 [==============================] - 0s 111us/sample - loss: 1.4313 - acc: 0.3238 - val_loss: 1.3565 - val_acc: 0.3556
Epoch 5/100
105/105 [==============================] - 0s 104us/sample - loss: 1.3958 - acc: 0.3238 - val_loss: 1.3247 - val_acc: 0.3556
Epoch 6/100
105/105 [==============================] - 0s 101us/sample - loss: 1.3618 - acc: 0.3238 - val_loss: 1.2959 - val_acc: 0.3556
Epoch 7/100
105/105 [==============================] -

### Model Prediction

In [89]:
model.predict(X2_test_array)

array([[0.40538815, 0.36303583, 0.23157594],
       [0.419497  , 0.36965063, 0.21085232],
       [0.3980898 , 0.3575214 , 0.24438877],
       [0.2629319 , 0.32094604, 0.41612205],
       [0.24998133, 0.31610286, 0.43391582],
       [0.4078006 , 0.36303824, 0.22916108],
       [0.25341076, 0.3175984 , 0.4289908 ],
       [0.26394978, 0.32145157, 0.4145986 ],
       [0.40533322, 0.36625546, 0.2284113 ],
       [0.41097146, 0.3661166 , 0.222912  ],
       [0.4110605 , 0.35774785, 0.23119162],
       [0.2790411 , 0.3279181 , 0.39304078],
       [0.24289466, 0.31109843, 0.4460069 ],
       [0.40538815, 0.36303583, 0.23157594],
       [0.25352383, 0.31853   , 0.42794624],
       [0.26983142, 0.3253841 , 0.4047845 ],
       [0.30566418, 0.3414015 , 0.35293433],
       [0.40876704, 0.3637856 , 0.22744742],
       [0.31496206, 0.34311768, 0.34192023],
       [0.26355863, 0.32007906, 0.41636232],
       [0.28219268, 0.331386  , 0.38642135],
       [0.269333  , 0.32473823, 0.40592873],
       [0.

### Save the Model

In [0]:
model.save('Iris.h5')

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer, loss='mse')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?